In [1]:
import pandas as pd
import get_data as gd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from os import walk

# Intro to the Data Sets

This notebook will introduce you to some foundational datasets for sabermetrics. An brief overview of the datasets, code to get the data, and some examples of how you can use the dataset to answer baseball questions will be provided.

The datasets introduced will be:
* The Lahman Dataset - It provides a large amount of season data from 1871 to the current season. It has an impressive amount of documentation and can be downloaded in a number of different sources (I use the csv version in this notebook)
* The Retro Dataset - Provides game by game or play by play data, a different zip file exists for each season. The data can be found here www.retrosheet.org/gamelogs/index.html.

## The Lahman Dataset

The section will go over (1) How to get the data to your local (2) Examples of questions you can answer with the data (3) Limitation of the dataset.

### Getting the Data
Neither function is to complex but both help save some time and manual steps. 

The first function "get_data" checks to see if a "master" folder already exists, if it doesn't it then uses the "dload" package to download and unzip all the files. Some potential good extensions to the get_data function would be to allow the user to provide the source link, name of the master file, and the option to update the content of the master file if it already exists.

The "make_df" function just saves you from having to remeber the location of the lahman dataset.

Again neither function is required, but makes life a litte bit easier.

In [ ]:
def get_data():
    """
    Function to download lahman data set, unzip and create a folder
    """
    if path.exists('master'):
        print('Folder already exists')
    else:
        dload.save_unzip("https://github.com/chadwickbureau/baseballdatabank/archive/master.zip")


def make_df(data_set: str) -> pd.DataFrame:
    """
    Simple function that saves writting out the file path
    :param data_set: Name of Lahman dataset you want
    :return: A dataframe of the dataset
    """
    assert isinstance(data_set, str)
    df: pd.DataFrame = pd.read_csv(f'master/baseballdatabank-master/core/{data_set}.csv')
    assert isinstance(df, pd.DataFrame)
    return df

### Example Questions & Limitation

Below are some example questions that will be answered, the Lahman dataset provides you with almost everything you need to answer season level questions. However, since all data is aggregated to a season you won't be able to dig into game/event specific questions such as "Do hot streaks exists for batter or pitchers?".

#### Q1 - What is the average number of home runs per game recorded in each decade? Does the rate of strikeouts show any correlation with the rate of home runs?

In [ ]:
teams_df = gd.make_df("Teams") # Get the Teams data 

# Convert the year into decades
teams_df['decade'] = round(teams_df['yearID']/10)*10 
# Use the groupby function to aggregate totals for G, HR and SO by decade
# I tend to find to easier to set as_index to false
decade_group = teams_df.groupby('decade', as_index=False)[['G','HR','SO']].sum()
#This removes double counting games
decade_group['G'] = decade_group['G']/2 
# Now you just need to calculate the averages by decade :)
decade_group['Avg HR Per Game'] = decade_group['HR']/decade_group['G']
decade_group['Avg SO Per Game'] = decade_group['SO']/decade_group['G']

# I will be using matplotlib, some other good libraries to check out are seaborn, plotly & altair
# As some future work I will re-make these graphs with the other libraries
# Style wise, I typically start with 538, then make some modification to make it with a white background
plt.style.use('fivethirtyeight')
fig = plt.figure(figsize=(20,15))
# Since we want to compare to different metrics over time I use the gricspec function
# Given the x-axis align and the y-axis don't I find it better to stack the graphs vertically rather than horizontally
# If they are side by side I find ppl tend not to notice the different in axis's 
# As a rule of thumb I try and always align axis of the same time, so in this case both graphs have x-axis that are by decade
gs = fig.add_gridspec(2,1)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])

fig.patch.set_facecolor('white')
ax1.set_facecolor('white')
ax1.plot(decade_group['decade'], decade_group['Avg HR Per Game'])
ax1.set_title('Average Home Runs Per Game')

ax2.set_facecolor('white')
ax2.plot(decade_group['decade'], decade_group['Avg SO Per Game'])
ax2.set_title('Average Strike Outs Per Game')
plt.show()

#### Q2 - What effect has the introduction of the Designated Hitter (DH) in the American League had in the difference in run scoring between the American and National Leagues?

In [ ]:
# Similiar to above you can do some simple groupping by league & year for games and runs
league_group = teams_df.groupby(['yearID','lgID'], as_index=False)[['G','R']].sum()

# I then seperate out NL & AL into their own dataframes to make graphing & calculations easier 
nl_group = league_group[league_group['lgID'] == 'NL']
al_group = league_group[league_group['lgID'] == 'AL']

# Find the average runs per game (since we seperated leagues no need to remove double counting)
nl_group['Avg Runs Per Game'] = nl_group['R']/nl_group['G']
al_group['Avg Runs Per Game'] = al_group['R']/al_group['G']

plt.style.use('fivethirtyeight')
# First attempt at graphing this I just plotted the two leagues over top of each other
# This seemed like the natural way to look at it but it was hard to tell what the impact was
fig, ax = plt.subplots(figsize= (20,10))
fig.patch.set_facecolor('white')
ax.plot(nl_group['yearID'], nl_group['Avg Runs Per Game'], label='National League')
ax.plot(al_group['yearID'], al_group['Avg Runs Per Game'], label='American League')
ax.set_facecolor('white')
ax.legend()
ax.set_title('Average Runs by League')
# draw a line for when the DH was added to make seeing the impact easier
# TODO Add some text indicating what this line is
plt.axvline(x=1973, color='black', linestyle='--') 

In [ ]:
# In my second attempt I merge the AL & NL sets together and then graph the difference
league_merge = pd.merge(nl_group, al_group, on='yearID',suffixes=('_nl', '_al'))
league_merge['Avg Run Difference'] = league_merge['Avg Runs Per Game_nl'] - league_merge['Avg Runs Per Game_al']

# The impact of the DH is more clear to see in this method
# The interesting thing to notice is how slowly how the impact of the DH has disappeared
# This makes me want to dig into what is causing the decline in the impact
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize= (20,10))
fig.patch.set_facecolor('white')
ax.plot(league_merge['yearID'], league_merge['Avg Run Difference'])
ax.set_facecolor('white')
#ax.legend()
plt.axvline(x=1973, color='black', linestyle='--')
plt.axhline(y=0, color='black') # Add in a solid line to spot the difference a bit easier
plt.show()

#### Q3 - How does the percentage of games completed by the starting pitcher from 2000 to 2010 compare to the percentage of games 100 years before?

In [ ]:
pitching_df = gd.make_df("Pitching") # Get the Pitching data 

# I slice the data into two different dataframes based on the years in question
# Best to make a function which takes date ranges to do this
pitching_df_2000_to_2010 = pitching_df[(pitching_df['yearID'] >= 2000) & (pitching_df['yearID'] <= 2010)]
# Use the sum function you can easily figure out the % completed for the decades
avg_comp_rate_2000_to_2010 = pitching_df_2000_to_2010['CG'].sum()/pitching_df_2000_to_2010['GS'].sum()
pitching_df_1900_to_1910 = pitching_df[(pitching_df['yearID'] >= 1900) & (pitching_df['yearID'] <= 1910)]
avg_comp_rate_1900_to_1910 = pitching_df_1900_to_1910['CG'].sum()/pitching_df_1900_to_1910['GS'].sum()

plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize= (20,10))
fig.patch.set_facecolor('white')
ax.bar(x = ['1900 to 1910', '2000 to 2010'],
      height=[avg_comp_rate_1900_to_1910*100, avg_comp_rate_2000_to_2010*100])
ax.set_facecolor('white')
ax.yaxis.set_major_formatter(PercentFormatter())
ax.set_ylim(0,100)
ax.set_title('The Average Pitcher Game Completion Rate of\n1900 to 1910 vs 2000 to 2010')
plt.ylabel('% of Game Completed (Pitchers)')
#ax.legend()
plt.show()

## The Retro Dataset

Like before this section will go over how to get the data and then some examples on what questions you can answer with the dataset.

### Getting the data

The first function will create a retro folder and then iterate through each year, download and unzip the file.

The second function will go through all the years and combined them into one single dataframe. 

In [ ]:
def get_retro_data():
    retro_path = './retro'
    os.mkdir(retro_path)
    for year in range(1871,2020):
        dload.save_unzip(f'https://www.retrosheet.org/gamelogs/gl{year}.zip', extract_path=retro_path , delete_after=True)
        
def make_gamelog_df():
    df = pd.read_csv('https://raw.githubusercontent.com/maxtoki/baseball_R/master/data/game_log_header.csv')
    cols = list(df.columns)
    for year in range(1871,2020):
        df_temp = pd.read_csv(f'retro/GL{year}.TXT',names=cols)
        df = df.append(df_temp, ignore_index=True)
    return df

In [ ]:
df = gd.make_gamelog_df()

df['HR'] = df['VisitorHR']+df['HomeHR']
df['GameCounter'] = 1
df['Month'] = df['Date'].apply(lambda x: str(x)[4:6])
df = df[~df['HR'].isnull()]
df_month = df.groupby('Month', as_index=False)[['HR','GameCounter']].apply(lambda x : x.astype(int).sum())
df_month['Avg Home Runs by Month'] = df_month['HR']/df_month['GameCounter']

plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize= (20,10))
fig.patch.set_facecolor('white')
ax.bar(x = df_month['Month'],
      height=df_month['Avg Home Runs by Month'])
ax.set_facecolor('white')
ax.set_title('What Month has the Most Home Runs?')
plt.ylabel('Average Home Runs Per Game by Month')
plt.show()

In [ ]:
park_data = pd.read_csv('https://www.retrosheet.org/parkcode.txt')
df['Year'] = df['Date'].apply(lambda x: str(x)[:4])
df_post_2000 = df[df['Year'].astype(int) >= 2000]

df_park = df_post_2000.groupby('ParkID', as_index=False)[['HR','GameCounter']].apply(lambda x : x.astype(int).sum())
df_park['Avg Home Runs by Park'] = df_park['HR']/df_park['GameCounter']

df_park_merged = pd.merge(df_park, park_data, left_on='ParkID', right_on='PARKID')
df_park_merged.sort_values(by = 'Avg Home Runs by Park', inplace=True)

plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize= (10,15))
fig.patch.set_facecolor('white')
ax.barh(y = df_park_merged['NAME'],
      width=df_park_merged['Avg Home Runs by Park'])
ax.set_facecolor('white')
ax.set_title('What Ballpark has the Most Home Runs?')
#plt.ylabel('Average Home Runs Per Game by Month')
plt.show()

#### Q - Do runs happen more frequently when some umpires are be hind the plate? What is the difference between the most pitcher friendly and the most hitter-friendly umpires?

In [ ]:
df_post_2000['Runs'] = df_post_2000['VisitorRunsScored']+df_post_2000['HomeRunsScore']
df_umpire_grouped = df_post_2000.groupby('UmpireHName', as_index=False)[['Runs', 'GameCounter']].apply(lambda x : x.astype(int).sum())
df_umpire_grouped['Runs Per Game'] = df_umpire_grouped['Runs']/df_umpire_grouped['GameCounter']

df_umpire_grouped = df_umpire_grouped[df_umpire_grouped['GameCounter'] > 400]
df_umpire_grouped.sort_values(by='Runs Per Game', inplace=True)

plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize= (10,15))
fig.patch.set_facecolor('white')
ax.barh(y = df_umpire_grouped['UmpireHName'],
      width=df_umpire_grouped['Runs Per Game'])
ax.set_facecolor('white')
ax.set_title('Which umpire allows the most runs?')
#plt.ylabel('Average Home Runs Per Game by Month')
plt.show()

#### Q - How many extra people attend ballgames during the weekend? What’s the average attendance by day of the week?

In [ ]:
df_day_of_week = df_post_2000.groupby('DayOfWeek',as_index=False)['Attendence'].mean()
day_of_week = {
    'Mon':1,
    'Tue':2,
    'Wed':3,
    'Thu':4,
    'Fri':5,
    'Sat':6,
    'Sun':7
}
df_day_of_week['DayOrdering'] = df_day_of_week['DayOfWeek'].map(day_of_week)
df_day_of_week.sort_values(by='DayOrdering',inplace=True)

plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize= (20,10))
fig.patch.set_facecolor('white')
ax.bar(x = df_day_of_week['DayOfWeek'],
      height=df_day_of_week['Attendence'])
ax.set_facecolor('white')
ax.set_title('What Day of the Week has the Highest Attendence?')
plt.ylabel('Average Attendence')
plt.show()

In [1]:
import dload
import pandas as pd
import os
import get_data as gd

In [4]:
gd.get_event_data()

In [6]:
import csv

def append_row(append_row, df, pkey):
    append_row[0]=pkey
    row_series = pd.Series(append_row, index = df.columns)
    df = df.append(row_series, ignore_index=True)
    return df

files = []
for (dirpath, dirnames, filenames) in walk('./game_event'):
    files.extend(filenames)
    
info_df = pd.DataFrame(columns = ['id','one', 'two'])
start_df = pd.DataFrame(columns = ['id','one', 'two', 'three', 'four', 'five'])
play_df = pd.DataFrame(columns = ['id','one', 'two', 'three', 'four', 'five', 'siz'])
com_df = pd.DataFrame(columns = ['id','one'])
sub_df = pd.DataFrame(columns = ['id','one', 'two', 'three', 'four', 'five'])
data_df = pd.DataFrame(columns = ['id','one', 'two', 'three'])
    
for file in files:
    with open(f'./game_event/{file}') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0

        for row in csv_reader:
            if row[0] == 'id':
                primary_key = row[1]

            elif row[0] =='info':
                info_df = append_row(row, info_df, primary_key)

            elif row[0] == 'start':
                start_df = append_row(row, start_df, primary_key)

            elif row[0] == 'play':
                play_df = append_row(row, play_df, primary_key)

            elif row[0] == 'com':
                com_df = append_row(row, com_df, primary_key)

            elif row[0] == 'sub':
                sub_df = append_row(row, sub_df, primary_key)

            elif row[0] == 'data':
                data_df = append_row(row, data_df, primary_key)

ValueError: Length of passed values is 6, index implies 3.

In [8]:
row

['CHN191604200', 'scorer', '?69', '109', '111', '485?']

In [9]:
info_df

,id,one,two
0,DET191604200,inputprogvers,version 7RS(19) of 07/07/92
1,DET191604200,visteam,CHA
2,DET191604200,hometeam,DET
3,DET191604200,date,1916/04/20
4,DET191604200,site,DET04
...,...,...,...
9089,CHN191604200,daynight,day
9090,CHN191604200,usedh,false
9091,CHN191604200,translator,Cramer/Herdman
9092,CHN191604200,inputter,Dick Cramer
